In [1]:
import more_utils
more_utils.set_logging_level("INFO")
import pyarrow
from more_utils.persistence import ModelarDB
from more_utils.time_series import ModelTable
import pyarrow as pa
from datetime import datetime

### Create connection to ModelarDB

Params:

1. hostname: ip adrress of the system where ModelarDB is running
2. interface: arrow (recommended), socket, http


In [2]:
conn_obj = ModelarDB.connect(
    hostname="localhost",
    manager_port=9998,
    edge_port=9999,
    cloud_port=9997,
    interface="arrow",
)

#### List tables in ModelarDB


In [3]:
conn_obj.list_tables()

['solar', 'solar2', 'windmill']

#### Create Model Table from Parquet file: from_parquet_file()

Params:

- modelardb_conn (ModelarDB): ModelarDB connection object
- file_path (str): parquet file path


In [4]:
model_table = ModelTable.from_parquet_file(conn_obj, "/home/dhaval/dev/datasets/solar.parquet")

#### Persist Table


In [5]:
model_table.persist(table_name="solar2", error_bound=0)

[2023-12-09 14:41:37:433] - INFO - moreutils (ModelTable) - Model Table 'solar2' created.


[2023-12-09 14:41:37:455] - INFO - moreutils (ModelTable) - Data inserted successfully into the table 'solar2'.


#### Apply Flush


In [4]:
conn_obj.flush(mode="FlushEdge")

[2023-12-09 14:52:35:909] - INFO - moreutils (ModelarDB) - FlushEdge: Compressed data buffers flushed.


#### Create custom data using Arrow Table format


In [5]:
# Read the data into a PyArrow table.
timestamp = pyarrow.array([
    pa.scalar(datetime(2023, 9, 10), type=pa.timestamp('us')), 
    pa.scalar(datetime(2023, 9, 12), type=pa.timestamp('us')), 
    pa.scalar(datetime(2023, 9, 14), type=pa.timestamp('us'))])
wind_turbine = pyarrow.array(["1026", "1026", "1026"])
wind_direction = pyarrow.array([300.0, 300.0, 300.0])
wind_speed = pyarrow.array([4.0, 4.0, 4.0])

names = ["timestamp", "wind_turbine", "wind_direction", "wind_speed"]
table = pyarrow.Table.from_arrays([timestamp, wind_turbine, wind_direction, wind_speed], names=names)

#### Create Model Table from arrow table: from_arrow_table()

Params:

- modelardb_conn (ModelarDB): ModelarDB connection object
- arrow_table (str): pyarrow table format

In [6]:
model_table = ModelTable.from_arrow_table(conn_obj, table)

#### Persist and flush


In [7]:
model_table.persist(table_name="test_table", error_bound=0)

[2023-12-09 14:54:31:699] - INFO - moreutils (ModelTable) - Model Table 'test_table' created.
[2023-12-09 14:54:31:710] - INFO - moreutils (ModelTable) - Data inserted successfully into the table 'test_table'.


#### Apply Flush


In [8]:
conn_obj.flush(mode="FlushEdge")

[2023-12-09 14:55:11:745] - INFO - moreutils (ModelarDB) - FlushEdge: Compressed data buffers flushed.
